In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_json)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/gemma2/keras/gemma2_2b_en/1/config.json
/kaggle/input/gemma2/keras/gemma2_2b_en/1/tokenizer.json
/kaggle/input/gemma2/keras/gemma2_2b_en/1/metadata.json
/kaggle/input/gemma2/keras/gemma2_2b_en/1/model.weights.h5
/kaggle/input/gemma2/keras/gemma2_2b_en/1/assets/tokenizer/vocabulary.spm
/kaggle/input/multilingual-idioms-indian/Gemma/burmese.json
/kaggle/input/multilingual-idioms-indian/Gemma/malayalam.json
/kaggle/input/multilingual-idioms-indian/Gemma/nepali.json
/kaggle/input/multilingual-idioms-indian/Gemma/catalan.json
/kaggle/input/multilingual-idioms-indian/Gemma/hindi.json
/kaggle/input/multilingual-idioms-indian/Gemma/croatian.json
/kaggle/input/multilingual-idioms-indian/Gemma/slovak.json
/kaggle/input/multilingual-idioms-indian/Gemma/finnish.json
/kaggle/input/multilingual-idioms-indian/Gemma/icelandic.json
/kaggle/input/multilingual-idioms-indian/Gemma/ukrainian.json
/kaggle/input/multilingual-idioms-indian/Gemma/thai.json
/kaggle/input/multilingual-idioms-indian

In [2]:
df= pd.read_json('/kaggle/input/multilingual-idioms-indian/Gemma/telugu.json')
df

,idiom,literal_meaning,figurative_meaning,example,language
0,గొంతు కట్టడం,To tie the throat.,To face a difficult situation bravely.,He had to tie his throat and speak up during t...,Telugu
1,చెట్టు కింద కూర్చోవడం,Sitting under a tree.,To take shelter or rest.,"After a long day, he just wanted to sit under ...",Telugu
2,వాటిని పక్కన పెట్టడం,Putting them aside.,Ignoring distractions or irrelevant matters.,"In order to focus, he decided to put those tho...",Telugu
3,నక్క కడుపులో నడవడం,Walking in the belly of a jackal.,Being in a dangerous or tricky situation.,He felt like he was walking in the belly of a ...,Telugu
4,కాళ్ళు కట్టడం,Tying the legs.,To restrict someone's freedom or movement.,The new rules are tying our legs and making it...,Telugu
5,కలలు కనడం,To see dreams.,To aspire or hope for something better.,She always sees dreams of becoming a doctor on...,Telugu
6,చెట్టు మీద ఎక్కడం,To climb the tree.,To take risks or venture into unknown territory.,He decided to climb the tree and start his own...,Telugu
7,అల్లుడు కొట్టడం,To hit the son-in-law.,To blame someone for problems they didn't cause.,Don't blame him for the failure; it's like hit...,Telugu
8,పచ్చి మిరపకాయలు,Green chilies.,Something that is fresh and lively.,"The festival was full of life, just like fresh...",Telugu
9,మంచి పనికి ముద్ర వేయడం,To stamp a good work.,To acknowledge or validate good work.,The manager stamped her good work with praise ...,Telugu


In [3]:
data= pd.read_json('/kaggle/input/multilingual-idioms-indian/Gemma/hindi.json')
data

,idiom,literal_meaning,figurative_meaning,example,language
0,नज़रबंद करना,To put under house arrest.,To restrict someone's freedom.,"Usne apne vicharon ko nazarband kiya, kyunki w...",Hindi
1,धरना देना,To sit in protest.,To stand firm on an issue.,Kisaan dharna dekar apni maangein manwane mein...,Hindi
2,दीवारों के कान होना,Walls have ears.,Confidential matters may be overheard.,"Jab bhi tum baat karo, yaad rakho deewaron ke ...",Hindi
3,थक कर चूर होना,To be completely exhausted.,To be extremely tired.,Kaam karte-karate woh thak kar choor ho gaya.,Hindi
4,तिनके का सहारा,A straw's support.,A small help in a difficult situation.,Uski madad tinke ka sahara saabit hui jab sab ...,Hindi
5,सांप भी मर जाए और लाठी भी न टूटे,"The snake should die, but the stick should not...",Finding a solution that satisfies all parties ...,Is samasya ka samadhan aisa hona chahiye ki sa...,Hindi
6,बंदर क्या जाने अदरक का,What does a monkey know of ginger?,Someone is unaware of the value or significanc...,Usne jab us kitaab ki keemat dekhi to mujhe la...,Hindi
7,छोटा मुँह बड़ी बात,"Small mouth, big talk.",Talking beyond one's status.,Uska chhota munh badi baat karna sabko chaukat...,Hindi
8,जख्म पर नमक छिड़कना,Sprinkling salt on wounds.,Adding to someone's pain.,Usne mere jakhm par namak chhidka jab usne mer...,Hindi
9,टक्कर खाना,To take a hit.,To compete or confront.,Bazaar mein pratispardha ke chalte humein takk...,Hindi


In [4]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

In [5]:
import os

os.environ['KERAS_BACKEND'] = 'jax'
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [6]:
import keras
import keras_nlp

In [7]:
from datasets import load_dataset

ds = load_dataset("json",data_files='/kaggle/input/multilingual-idioms-indian/Gemma/*.json')

Resolving data files:   0%|          | 0/71 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
#Display dataset summary
if 'ds' in locals():
    print(ds)

DatasetDict({
    train: Dataset({
        features: ['idiom', 'literal_meaning', 'figurative_meaning', 'example', 'language'],
        num_rows: 720
    })
})


In [9]:
from datasets import DatasetDict
# Initialize an empty list to store the formatted examples
data = []
# Access the 'train' split of your dataset
train_data = ds["train"]

# Add debug printing to see what fields are available
print("First example keys:", list(train_data[0].keys()))

# Iterate over each example in the dataset
for i, example in enumerate(train_data):
    try:
        # Check if required fields are available and valid
        required_fields = ["idiom", "literal_meaning", "figurative_meaning", "example"]
        
        # Print missing fields for debugging
        missing_fields = [field for field in required_fields if field not in example]
        if missing_fields:
            print(f"Example {i} is missing fields: {missing_fields}")
            continue
            
        # Create a template with instruction and response format
        template = (
        "Instruction:\n"
        "Find a suitable idiom for this situation: {figurative_meaning}\n\n"
        "Response:\n"
        "Original Idiom ({language}): {idiom}\n"
        "English Translation: {literal_meaning}\n"
        "Example Usage: {example}\n"
        "Cultural Context: This idiom comes from {language} and is commonly used when {figurative_meaning}.\n"
        )
        
        # Format the example and add it to the data list
        formatted_example = template.format(**example)
        data.append(formatted_example)
        
    except KeyError as e:
        print(f"KeyError in example {i}: {str(e)}")
        print(f"Available keys: {list(example.keys())}")
        continue

# Limit to the first 1000 examples
data = data[:1000]

# Display the first few formatted examples
for i, example in enumerate(data[:5]):
    print(f"Example {i + 1}:\n{example}\n")

First example keys: ['idiom', 'literal_meaning', 'figurative_meaning', 'example', 'language']
Example 1:
Instruction:
Find a suitable idiom for this situation: Exaggerating a small issue.

Response:
Original Idiom (Arabic): اجعل من الحبة قبة
English Translation: Make a dome from a grain.
Example Usage: He tends to make a dome from a grain when discussing minor problems.
Cultural Context: This idiom comes from Arabic and is commonly used when Exaggerating a small issue..


Example 2:
Instruction:
Find a suitable idiom for this situation: It's better to have a small but certain advantage than the possibility of a greater one.

Response:
Original Idiom (Arabic): عصفور في اليد خير من عشرة على الشجرة
English Translation: A bird in the hand is better than ten in the tree.
Example Usage: Don't risk your stable job for uncertain opportunities; remember, a bird in the hand is better than ten in the tree.
Cultural Context: This idiom comes from Arabic and is commonly used when It's better to hav

In [10]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_2b_en")
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,617,270,528 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,617,270,528 (9.75 GB)

 Trainable params: 2,928,640 (11.17 MB)

 Non-trainable params: 2,614,341,888 (9.74 GB)

In [12]:
# Limit the input sequence length to 256 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 256
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=2, batch_size=1)

Epoch 1/2
720/720 ━━━━━━━━━━━━━━━━━━━━ 344s 442ms/step - loss: 0.6049 - sparse_categorical_accuracy: 0.6729
Epoch 2/2
720/720 ━━━━━━━━━━━━━━━━━━━━ 319s 419ms/step - loss: 0.3168 - sparse_categorical_accuracy: 0.8146


In [13]:
#Define figurative meaning to be tested
test_meaning = "to be stuck in a difficult situation"


#Using the same template format as training
prompt = (
    "Instruction:\n Which  idiom would be suitable for {}\n\n"
    "Response:\n"
).format(test_meaning)


sampler = keras_nlp.samplers.TopKSampler(k=10, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=512))

Instruction:
 Which  idiom would be suitable for to be stuck in a difficult situation

Response:
The idiom that would be suitable for to be stuck in a difficult situation is  to be caught between a rock and a hard place.

Explanation:
In this situation, you are between two unpleasant options and you need to make a choice. The idiom captures this feeling accurately.
Example Usage: He found himself caught between a rock and a hard place; he had to decide which option to choose.
Cultural Context: This idiom is commonly used in English and is appropriate for this context.



In [14]:
#Define figurative meaning to be tested
test_meaning = "He who does not know you does not value you"


#Using the same template format as training
prompt = (
    "Instruction:\n Which  idiom would be suitable for {}\n\n"
    "Response:\n"
).format(test_meaning)


sampler = keras_nlp.samplers.TopKSampler(k=10, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=512))

Instruction:
 Which  idiom would be suitable for He who does not know you does not value you

Response:
The idiom that would be suitable for He who does not know you does not value you is He who does not know does not appreciate.

Explanation:
This idiom comes from English and refers to the lack of knowledge or understanding. It can be used when He who does not know you does not value you.
Example Usage: He who does not know you does not appreciate your work.
Cultural Context: This idiom is commonly used in English and has a general context in various situations.



In [15]:
#Define figurative meaning to be tested
test_meaning = "to be indecisive or confused"


#Using the same template format as training
prompt = (
    "Instruction:\n Which  idiom would be suitable for {}\n\n"
    "Response:\n"
).format(test_meaning)


sampler = keras_nlp.samplers.TopKSampler(k=10, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=512))

Instruction:
 Which  idiom would be suitable for to be indecisive or confused

Response:
The idiom "to count chickens before they are hatched" would be suitable.

Explanation:
This idiom comes from Latin and means "to count chickens before they are hatched; to assume too much certainty prematurely." It is commonly used when someone is too confident or overzealous.
Example Usage: After his impressive performance at the tournament, he started counting his chickens before they were hatched.
Original Sentence: Don't count your chickens before they are hatched; take it one step at a time.

